In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import gensim.downloader as api
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from scipy.spatial.distance import cosine

# Charger un modèle Word2Vec pré-entraîné en anglais
model = api.load("glove-wiki-gigaword-100")  # 100 dimensions GloVe model
stop_words = set(stopwords.words('english'))

# Fonction de tokenisation et de filtrage des mots d'arrêt
def preprocess(sentence):
    tokens = word_tokenize(sentence.lower())
    return [word for word in tokens if word.isalpha() and word not in stop_words]


In [3]:
# Fonction pour calculer le vecteur de contexte moyen
def get_context_vector(words, target_index):
    context_vectors = []
    for i, word in enumerate(words):
        if i != target_index and word in model:
            context_vectors.append(model[word])
    return np.mean(context_vectors, axis=0) if context_vectors else None

# Fonction de détection des erreurs sémantiques
def detect_typo_semantic_error(sentence, threshold=0.5):
    words = preprocess(sentence)
    errors = []

    for i, word in enumerate(words):
        if word not in model:
            continue
        context_vector = get_context_vector(words, i)
        if context_vector is not None:
            word_vector = model[word]
            similarity = 1 - cosine(word_vector, context_vector)

            # Si la similarité est inférieure au seuil, on considère qu'il y a une erreur
            if similarity < threshold:
                errors.append((word, similarity))

    return errors


In [4]:
# Exemples de phrases avec et sans erreurs sémantiques
sentences = [
    "The cat sat on the mat.",          # Pas d'erreur
    "The cat sat on the hat.",          # Erreur (hat au lieu de mat)
    "I put a nice cat on my head.",     # Erreur (head au lieu de hat)
    "She bought a book.",               # Pas d'erreur
    "She bought a boot."                # Erreur (boot au lieu de book)
]

# Tester les phrases
for sentence in sentences:
    errors = detect_typo_semantic_error(sentence)
    if errors:
        print(f"Erreur détectée dans la phrase: '{sentence}'")
        for word, similarity in errors:
            print(f" - Mot suspecté: '{word}' avec similarité {similarity:.2f}")
    else:
        print(f"Aucune erreur détectée dans la phrase: '{sentence}'")


Erreur détectée dans la phrase: 'The cat sat on the mat.'
 - Mot suspecté: 'cat' avec similarité 0.24
 - Mot suspecté: 'sat' avec similarité 0.37
 - Mot suspecté: 'mat' avec similarité 0.32
Erreur détectée dans la phrase: 'The cat sat on the hat.'
 - Mot suspecté: 'cat' avec similarité 0.46
 - Mot suspecté: 'sat' avec similarité 0.32
Erreur détectée dans la phrase: 'I put a nice cat on my head.'
 - Mot suspecté: 'cat' avec similarité 0.47
Erreur détectée dans la phrase: 'She bought a book.'
 - Mot suspecté: 'bought' avec similarité 0.39
 - Mot suspecté: 'book' avec similarité 0.39
Erreur détectée dans la phrase: 'She bought a boot.'
 - Mot suspecté: 'bought' avec similarité 0.13
 - Mot suspecté: 'boot' avec similarité 0.13
